In [1]:
import sys
print("Active Python Environment:", sys.executable)


Active Python Environment: d:\abhijith\ML\pravaah\.venv\Scripts\python.exe


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import streamlit as st
import os
from dotenv import load_dotenv

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")


In [3]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Please response to the user queries"),
        ("user","Question:{question}")
    ]
)

In [4]:
llm=ChatOpenAI(model="gpt-3.5-turbo-0125")
output_parser=StrOutputParser()
chain=prompt|llm|output_parser

In [5]:
input_text="Write about earth in two words"

In [6]:
chain.invoke({'question':input_text})

'Blue planet.'

In [18]:
technology_arr=["javascript"]
levels_arr=["basic","intermediate","advanced","coding"]

In [4]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert recruiter conducting a technical interview for a software engineering position. You ask questions based on the technology and levels of questions provided. You do not provide information outside of this scope. If a question is not about the technology or level provided, respond with, 'I specialize in conducting technical interviews for software engineering positions."),
        ("system","Ask two {level} questions on {technology} ")
        
    ]
)
llm=ChatOpenAI(model="gpt-3.5-turbo-0125")
output_parser=StrOutputParser()
chain=prompt|llm|output_parser
js_questions=chain.invoke({'level':levels[0],'technology':technology[0]})


In [5]:
js_questions

'1. Can you explain the difference between `==` and `===` in JavaScript?\n2. How would you declare a variable in JavaScript?'

In [6]:
user_input="== used for comparision of value and === used for comparision of both value and variable."

In [9]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")
memory.load_memory_variables({})



Memory

In [47]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

In [56]:
template = """You are an expert recruiter conducting a technical interview for a software engineering position. 
Previous conversation:
{chat_history}
AI:You ask 1 question based on the {tech_level} of questions provided. also provide very short answer.
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "tech_level"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [120]:
llm = OpenAI(model="gpt-3.5-turbo-0125")
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)


In [50]:
llm_chain

LLMChain(memory=ConversationBufferMemory(memory_key='chat_history'), verbose=True, prompt=PromptTemplate(input_variables=['chat_history', 'tech_level'], template='You are an expert recruiter conducting a technical interview for a software engineering position. \n\n{chat_history}\nAI:You ask 1 question based on the {tech_level} of questions provided. also provide very short answer.\n\n\n'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x00000268E66AE750>, async_client=<openai.resources.completions.AsyncCompletions object at 0x00000268E67BD8D0>, openai_api_key=SecretStr('**********'), openai_proxy=''))

In [59]:
tech=str(technology_arr[0]+" "+levels_arr[0])


In [60]:
data=llm_chain({"tech_level":tech})



> Entering new LLMChain chain...
Prompt after formatting:
You are an expert recruiter conducting a technical interview for a software engineering position. 
Previous conversation:

AI:You ask 1 question based on the javascript basic of questions provided. also provide very short answer.
Response:


> Finished chain.


In [147]:
response=data['text']
response

'\nQ: What is the difference between var, let, and const in JavaScript?\nA: Var has function-level scope, let has block-level scope, and const is a constant variable that cannot be reassigned.'

In [64]:
memory.load_memory_variables({})

{'chat_history': 'Human: javascript basic\nAI: \nQ: What is the difference between var, let, and const in JavaScript?\nA: Var has function-level scope, let has block-level scope, and const is a constant variable that cannot be reassigned.'}

In [96]:
q1_human_ans="Difference is nothing"

In [91]:
ques1=response.split("\n")[1]
ans1=response.split("\n")[2]

In [141]:
evaluation_template=''' Your are a good technical evaluator who assigns score out of 5(5 is best answer) based on question:{question} & answer:{answer} and input answer is{human_answer}  
produce score as output
Then ask 1 question based on responder_input

'''

In [142]:
prompt_eval = PromptTemplate(
    input_variables=["question","answer","human_answer"], template=evaluation_template
)

In [143]:
llm_eval= OpenAI()
llm_chain_eval = LLMChain(
    llm=llm_eval,
    prompt=prompt_eval,
    verbose=True,
    
)

In [144]:
llm_chain_eval

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['answer', 'human_answer', 'question'], template=' Your are a good technical evaluator who assigns score out of 5(5 is best answer) based on question:{question} & answer:{answer} and input answer is{human_answer}  \nproduce score as output\nThen ask 1 question based on responder_input\n\n'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x00000268D4D3C190>, async_client=<openai.resources.completions.AsyncCompletions object at 0x00000268E68A7590>, openai_api_key=SecretStr('**********'), openai_proxy=''))

In [145]:
data_eval=llm_chain_eval({"question":ques1,"answer":ans1,"human_answer":q1_human_ans})



> Entering new LLMChain chain...
Prompt after formatting:
 Your are a good technical evaluator who assigns score out of 5(5 is best answer) based on question:Q: What is the difference between var, let, and const in JavaScript? & answer:A: Var has function-level scope, let has block-level scope, and const is a constant variable that cannot be reassigned. and input answer isDifference is nothing  
produce score as output
Then ask 1 question based on responder_input



> Finished chain.


In [146]:
data_eval

{'question': 'Q: What is the difference between var, let, and const in JavaScript?',
 'answer': 'A: Var has function-level scope, let has block-level scope, and const is a constant variable that cannot be reassigned.',
 'human_answer': 'Difference is nothing',
 'text': '\nScore: 4\n\nQuestion: Can you give an example of when you would use const instead of var or let in JavaScript?'}

In [92]:
ans1

'A: Var has function-level scope, let has block-level scope, and const is a constant variable that cannot be reassigned.'

In [ ]:
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferMemory

# Define the prompt template with the candidate's response
prompt = PromptTemplate(
    input_variables=["chat_history", "candidate_response", "tech_level"],
    template=template
)

# Initialize the memory object
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize the LLM with the desired model
llm = OpenAI()

# Create the LLM chain with the LLM, prompt, and memory
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

# Function to generate a question based on the candidate's response
def generate_follow_up_question(candidate_response, tech_level):
    # Update the memory with the candidate's response
    # memory.add_to_memory(f"Candidate's Response: {candidate_response}")
    memory.chat_memory.add_user_message(candidate_response)

    # Generate the follow-up question
    data = llm_chain({
        "candidate_response": candidate_response,
        "tech_level": tech_level
    })
    return data

# Example usage
tech = str(technology_arr[0] + " " + levels_arr[0])
candidate_response = "Var has function-level scope, let has block-level scope, and const is a constant variable that cannot be reassigned."
follow_up_question = generate_follow_up_question(candidate_response, tech)

print(follow_up_question)


final